# Radius Search Practice
To be used for bare critical mass calculation
<br >Created on 18 October 2022

In [1]:
# prepare the environment
import openmc
import openmc.model
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

## Create Parametrized Model

In [11]:
def build_model(radius):
    
    # define materials
    # HEU U.S. Average taken from PNNL Material compendium
    fuel = openmc.Material(name="Bare Sphere")
    fuel.set_density('g/cc', 18.724760)
    fuel.add_nuclide('U234', 0.009849, 'ao')
    fuel.add_nuclide('U235', 0.931550, 'ao')
    fuel.add_nuclide('U236', 0.004500, 'ao')
    fuel.add_nuclide('U238', 0.054150, 'ao')
    
    materials = openmc.Materials([fuel])
    
    # create sphere with radius parameter
    sphere_radius = openmc.Sphere(x0=0,y0=0,z0=0,r=radius, boundary_type='vacuum', name='BCM')
    
    # create core cell
    core_cell = openmc.Cell(name='Bare Critical Sphere')
    core_cell.fill = fuel
    core_cell.region = -sphere_radius
    
    # create universe geometry
    root_uni = openmc.Universe()
    root_uni.add_cells([core_cell])
    
    geometry = openmc.Geometry(root_uni);
    # define criticality settings
    settings = openmc.Settings()
    settings.run_mode = 'eigenvalue' # keff calculation
    settings.particles = 5000 # particles per batch (mo betta)
    settings.batches = 250 # number of batches
    settings.inactive = 50 # inactive batches
    
    bounds = [-radius, -radius, -radius, 
             radius, radius, radius]
    uniform_dist = openmc.stats.Box(bounds[:3],bounds[3:],
                                   only_fissionable=True)
    settings.source = openmc.Spurce(space=uniform_dist)
    settings.output = {'tallies': False}
    
    model = openmc.model.Model(bracket[1,100],geometry,materials,settings)
    
    return model

## Conduct Radius Search

In [12]:
crit_r, guesses, keffs = openmc.search_for_keff(build_model, 
                                                tol=1e-3, print_iterations=True,
                                               run_args={'output':False})
print('The bare critical sphere radius is %7.4f cm.' % crit_r)

UnboundLocalError: local variable 'model' referenced before assignment